# Claude Code Colab Bootstrap

**What this notebook does:**
- Installs Claude Code in your Colab environment
- Writes a setup guide, skills, and agents that Claude can read
- Generates a bootstrap prompt to configure Claude for your project

**How to use it:**
1. **Get your auth token** (see cell below) and save it to Colab Secrets
2. Run all cells → open terminal → run `claude` → paste bootstrap prompt
3. Claude reads the guide and sets up your environment
4. Claude can edit `bootstrap_config.json` and `_bootstrap_source.ipynb` to improve future bootstraps

> **Ephemeral by default:** No Google Drive access needed. Just re-run the notebook each session.
> Check `USE_GOOGLE_DRIVE` below if you want persistence between sessions.

---

## 🔑 Authentication Setup

**To get a long-lived Claude Code OAuth token:**

1. On your local machine (laptop/desktop), run:
   ```bash
   claude setup-token
   ```
2. Copy the token that's printed
3. In Colab, click the **key icon** (🔑) in the left sidebar
4. Click **"Add secret"**
5. Name: `CLAUDE_CODE_OAUTH_TOKEN`
6. Value: Paste your token
7. Click **"Add"**

To verify, run this in a Python cell:
```python
from google.colab import userdata
userdata.get("CLAUDE_CODE_OAUTH_TOKEN")
```

The notebook will automatically use this token to authenticate Claude Code.

---

In [ ]:
# @title 1. Project Settings { display-mode: "form" }

# @markdown ### Storage Mode
USE_GOOGLE_DRIVE = False  # @param {type:"boolean"}
# @markdown - **Unchecked (default):** Ephemeral — no Drive access, resets each session
# @markdown - **Checked:** Persistent — saves to Google Drive

# @markdown ---
# @markdown ### Project
PROJECT_NAME = "my-project"  # @param {type:"string"}
PROJECT_TYPE = "ML/AI Development"  # @param ["ML/AI Development", "Data Science/Analysis", "Web Development", "General Python", "Custom"]
PROJECT_DESCRIPTION = ""  # @param {type:"string"}

# Derived paths
if USE_GOOGLE_DRIVE:
    WORKSPACE_PATH = f"/content/drive/My Drive/claude-workspaces/{PROJECT_NAME}"
    STORAGE_MODE = "persistent (Google Drive)"
else:
    WORKSPACE_PATH = f"/content/claude-workspaces/{PROJECT_NAME}"
    STORAGE_MODE = "ephemeral (resets each session)"

CLAUDE_CONFIG_PATH = f"{WORKSPACE_PATH}/.claude"

print(f"Project: {PROJECT_NAME}")
print(f"Storage: {STORAGE_MODE}")
print(f"Workspace: {WORKSPACE_PATH}")

In [ ]:
# @title 2. Mount Google Drive (if enabled)

if USE_GOOGLE_DRIVE:
    from google.colab import drive

    print("Mounting Google Drive...")
    drive.mount("/content/drive")
    print("✓ Drive mounted")
else:
    print("⏭️ Skipping Drive mount (ephemeral mode)")

In [ ]:
# @title 3. Install Claude Code & Dependencies
import json as json_lib
import os
import shutil
import subprocess
import sys

print("=" * 50)
print("Installing dependencies...")
print("=" * 50)

# Sandbox deps
print("\n[1/4] Sandbox dependencies...")
subprocess.run(
    "apt-get update -qq && apt-get install -qq -y socat bubblewrap > /dev/null 2>&1", shell=True
)
print("  ✓ socat, bubblewrap")

# Dev tools
print("\n[2/4] Python dev tools...")
subprocess.run("pip install -q black isort", shell=True)
print("  ✓ black, isort")

# Claude Code
print("\n[3/4] Claude Code...")
result = subprocess.run(
    "curl -fsSL https://claude.ai/install.sh | bash", shell=True, capture_output=True, text=True
)
if result.returncode == 0:
    print("  ✓ Claude Code installed")
else:
    print(f"  ⚠️ Install issue: {result.stderr[:200]}")
    print("  Retrying...")
    subprocess.run("curl -fsSL https://claude.ai/install.sh | bash", shell=True)

# Configure environment
print("\n[4/4] Configuring environment...")
bashrc_path = os.path.expanduser("~/.bashrc")
with open(bashrc_path, "r") as f:
    bashrc_content = f.read()
if ".local/bin" not in bashrc_content:
    with open(bashrc_path, "a") as f:
        f.write("\n# Claude Code\n")
        f.write('export PATH="$HOME/.local/bin:$HOME/.claude/bin:$PATH"\n')
        f.write("export TERM=xterm-256color\n")
        f.write("export COLORTERM=truecolor\n")
        f.write("export FORCE_COLOR=1\n")
print("  ✓ PATH and terminal configured")

# Create workspace
os.makedirs(WORKSPACE_PATH, exist_ok=True)
os.makedirs(CLAUDE_CONFIG_PATH, exist_ok=True)
print(f"  ✓ Workspace: {WORKSPACE_PATH}")

# Symlink ~/.claude
home_claude = os.path.expanduser("~/.claude")
if os.path.islink(home_claude):
    os.unlink(home_claude)
elif os.path.exists(home_claude):
    if os.path.isdir(home_claude):
        shutil.rmtree(home_claude)
    else:
        os.unlink(home_claude)
os.symlink(CLAUDE_CONFIG_PATH, home_claude)
print(f"  ✓ ~/.claude → {CLAUDE_CONFIG_PATH}")

print("\n" + "=" * 50)
print("✅ Installation complete!")
print("=" * 50)

In [ ]:
# @title 4. Configure Authentication

print("Configuring authentication...")

# Try to get token from Colab Secrets
oauth_token = None
try:
    from google.colab import userdata

    try:
        oauth_token = userdata.get("CLAUDE_CODE_OAUTH_TOKEN")
        if oauth_token:
            os.environ["CLAUDE_CODE_OAUTH_TOKEN"] = oauth_token
            print("  ✓ Found CLAUDE_CODE_OAUTH_TOKEN in Colab Secrets")
    except Exception as e:
        print(f"  ⚠️ Could not read CLAUDE_CODE_OAUTH_TOKEN: {e}")

    # Backwards compatibility
    if not oauth_token:
        try:
            oauth_token = userdata.get("CLAUDE_CODE_TOKEN")
            if oauth_token:
                os.environ["CLAUDE_CODE_OAUTH_TOKEN"] = oauth_token
                print("  ✓ Found CLAUDE_CODE_TOKEN in Colab Secrets")
                print("  ⚠️ Consider renaming it to CLAUDE_CODE_OAUTH_TOKEN")
        except Exception as e:
            print(f"  ⚠️ Could not read CLAUDE_CODE_TOKEN: {e}")
except ImportError:
    print("  ⚠️ Not in Colab environment")

# Also check environment variable
if not oauth_token:
    oauth_token = os.environ.get("CLAUDE_CODE_OAUTH_TOKEN")
    if oauth_token:
        print("  ✓ Found CLAUDE_CODE_OAUTH_TOKEN in environment")

# Write credentials if we have a token
if oauth_token:
    creds_path = f"{CLAUDE_CONFIG_PATH}/.credentials.json"
    creds = {"claudeAiOauth": {"accessToken": oauth_token}}
    with open(creds_path, "w") as f:
        json_lib.dump(creds, f)
    print(f"  ✓ Wrote credentials to {creds_path}")
    print("  ✓ Authentication configured!")
else:
    print("\n" + "!" * 60)
    print("⚠️  No Claude Code OAuth token found")
    print("Add this Colab Secret: CLAUDE_CODE_OAUTH_TOKEN")
    print("Generate it with: claude setup-token")
    print("Verify in a Python cell:")
    print("from google.colab import userdata")
    print("userdata.get('CLAUDE_CODE_OAUTH_TOKEN')")
    print("!" * 60 + "\n")
    print("📝 See the authentication setup cell above for instructions.")
    print("💡 You can also authenticate manually in the terminal with: claude")

In [ ]:
# @title 5. Capture Environment
import json as json_lib

print("Capturing environment snapshot...")

env_snapshot = {
    "storage_mode": "persistent" if USE_GOOGLE_DRIVE else "ephemeral",
    "workspace_path": WORKSPACE_PATH,
    "project_name": PROJECT_NAME,
    "project_type": PROJECT_TYPE,
    "python_version": sys.version.split()[0],
}

# GPU
try:
    import torch

    env_snapshot["gpu_available"] = torch.cuda.is_available()
    env_snapshot["pytorch_version"] = torch.__version__
    if torch.cuda.is_available():
        env_snapshot["gpu_name"] = torch.cuda.get_device_name(0)
        env_snapshot["gpu_memory_gb"] = round(
            torch.cuda.get_device_properties(0).total_memory / 1e9, 1
        )
        print(f"  GPU: {env_snapshot['gpu_name']} ({env_snapshot['gpu_memory_gb']}GB)")
    else:
        print("  ⚠️ No GPU (Runtime → Change runtime type → GPU)")
except Exception:
    env_snapshot["gpu_available"] = False

# Drive
drive_mounted = os.path.exists("/content/drive/My Drive") or os.path.exists(
    "/content/drive/MyDrive"
)
env_snapshot["drive_mounted"] = drive_mounted
if USE_GOOGLE_DRIVE and not drive_mounted:
    print("  ⚠️ Drive not mounted!")

# Tools
env_snapshot["tools"] = {}
for tool in ["claude", "black", "isort", "ruff", "socat", "bwrap", "git"]:
    r = subprocess.run(f"which {tool}", shell=True, capture_output=True)
    env_snapshot["tools"][tool] = r.returncode == 0

# Write
with open(f"{WORKSPACE_PATH}/ENVIRONMENT.json", "w") as f:
    json_lib.dump(env_snapshot, f, indent=2)

print("\n✓ Saved to ENVIRONMENT.json")

In [ ]:
# @title 6. Install Hooks & Status Line

print("Installing hooks and status line...")

# Create hooks directory
hooks_dir = f"{CLAUDE_CONFIG_PATH}/hooks"
os.makedirs(hooks_dir, exist_ok=True)

# Install markdown formatter hook
markdown_formatter = '''{{MARKDOWN_FORMATTER_HOOK}}'''
with open(f"{hooks_dir}/markdown_formatter.py", "w") as f:
    f.write(markdown_formatter)
os.chmod(f"{hooks_dir}/markdown_formatter.py", 0o755)
print("  ✓ markdown_formatter.py")

# Install status line script
statusline_script = '''{{STATUSLINE_HOOK}}'''
with open(f"{hooks_dir}/statusline.py", "w") as f:
    f.write(statusline_script)
os.chmod(f"{hooks_dir}/statusline.py", 0o755)
print("  ✓ statusline.py")

# Create/update settings.json with hooks and statusline
settings_path = f"{CLAUDE_CONFIG_PATH}/settings.json"
settings = {}
if os.path.exists(settings_path):
    with open(settings_path, "r") as f:
        settings = json_lib.load(f)

# Add hooks configuration
settings["hooks"] = {
    "PostToolUse": [
        {
            "matcher": "Edit|Write",
            "hooks": [{"type": "command", "command": f'"{hooks_dir}/markdown_formatter.py"'}],
        }
    ]
}

# Add status line configuration
settings["statusLine"] = {"type": "command", "command": f"{hooks_dir}/statusline.py"}

with open(settings_path, "w") as f:
    json_lib.dump(settings, f, indent=2)

print("  ✓ settings.json configured with hooks and statusline")
print("\n✓ Hooks and status line installed")

In [ ]:
# @title 7. Update Documentation (Optional)

# @markdown Update cached Claude Code documentation from the official site.
# @markdown This downloads the latest docs and adds metadata headers.

import json
import os
import subprocess

# Embed update script
update_docs_script = '''{{UPDATE_DOCS_SCRIPT}}'''

# Embed docs manifest
docs_manifest = {{DOCS_MANIFEST_JSON}}

# Create cached_docs directory in workspace
cached_docs_dir = f"{WORKSPACE_PATH}/cached_docs"
os.makedirs(cached_docs_dir, exist_ok=True)

# Write script to workspace
script_path = f"{WORKSPACE_PATH}/update_docs.py"
with open(script_path, "w") as f:
    f.write(update_docs_script)
os.chmod(script_path, 0o755)
print("✓ Wrote update_docs.py to workspace")

# Write manifest to workspace
manifest_path = f"{WORKSPACE_PATH}/docs_manifest.json"
with open(manifest_path, "w") as f:
    json.dump(docs_manifest, f, indent=2)
print("✓ Wrote docs_manifest.json to workspace")

# Run update
print("\nUpdating documentation from https://code.claude.com/docs/en/...")
result = subprocess.run(
    f'cd "{WORKSPACE_PATH}" && python3 update_docs.py', shell=True, capture_output=True, text=True
)
print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

print("\n✓ Documentation update complete")
print(f"\nDocs cached in: {cached_docs_dir}")
print("To update again, re-run this cell.")

In [ ]:
# @title 8. Write Guide & Config Files

# Guide
GUIDE = """{{GUIDE_CONTENT}}"""

with open(f"{WORKSPACE_PATH}/CLAUDE_CODE_COLAB_GUIDE.md", "w") as f:
    f.write(GUIDE)
shutil.copy(
    f"{WORKSPACE_PATH}/CLAUDE_CODE_COLAB_GUIDE.md",
    os.path.expanduser("~/CLAUDE_CODE_COLAB_GUIDE.md"),
)
print("✓ Guide written")

# Bootstrap config
config = {
    "_description": "Claude can edit this to customize future bootstraps",
    "project_name": PROJECT_NAME,
    "project_type": PROJECT_TYPE,
    "project_description": PROJECT_DESCRIPTION,
    "use_google_drive": USE_GOOGLE_DRIVE,
    "custom_packages": [],
    "custom_claude_md_sections": [],
}
with open(f"{WORKSPACE_PATH}/bootstrap_config.json", "w") as f:
    json_lib.dump(config, f, indent=2)
print("✓ Config written")

# Copy this notebook
import glob

notebook_files = glob.glob("/content/*.ipynb")
if notebook_files:
    shutil.copy(notebook_files[0], f"{WORKSPACE_PATH}/_bootstrap_source.ipynb")
    print("✓ Notebook copied to _bootstrap_source.ipynb")

In [ ]:
# @title 9. Install Skills & Agents

print("Installing skills and agents...")

skills_dir = f"{WORKSPACE_PATH}/.claude/skills"
agents_dir = f"{WORKSPACE_PATH}/.claude/agents"
os.makedirs(skills_dir, exist_ok=True)
os.makedirs(agents_dir, exist_ok=True)

SKILLS = {{SKILLS_DICT}}

for name, content in SKILLS.items():
    skill_path = f"{skills_dir}/{name}"
    os.makedirs(skill_path, exist_ok=True)
    with open(f"{skill_path}/SKILL.md", "w") as f:
        f.write(content)
    print(f"  ✓ Skill: {name}")

AGENTS = {{AGENTS_DICT}}

for name, content in AGENTS.items():
    with open(f"{agents_dir}/{name}.md", "w") as f:
        f.write(content)
    print(f"  ✓ Agent: {name}")

print(f"\n✓ Installed {len(SKILLS)} skills, {len(AGENTS)} agents")

In [ ]:
# @title 10. Generate Bootstrap Prompt { display-mode: "form" }

# @markdown ### What should Claude set up?
create_claude_md = True  # @param {type:"boolean"}
create_settings = True  # @param {type:"boolean"}
create_custom_commands = True  # @param {type:"boolean"}
run_diagnostics = True  # @param {type:"boolean"}

# Build prompt
if USE_GOOGLE_DRIVE:
    storage_note = f"Workspace: {WORKSPACE_PATH} (persistent on Drive)"
else:
    storage_note = f"Workspace: {WORKSPACE_PATH} (ephemeral - resets each session)"

prompt_parts = [
    f"Hi Claude! I'm setting up Claude Code in Google Colab for {PROJECT_TYPE}.",
    "",
    f"**Environment:** {storage_note}",
    "",
    "Please read these files first:",
    "- ENVIRONMENT.json (runtime snapshot)",
    "- CLAUDE_CODE_COLAB_GUIDE.md (setup guide)",
    "- bootstrap_config.json (editable config)",
    "",
    "**Please create:**",
]

if create_claude_md:
    prompt_parts.append(f"- CLAUDE.md with {PROJECT_TYPE} conventions and Colab notes")
if create_settings:
    prompt_parts.append("- .claude/settings.json with sensible defaults")
if create_custom_commands:
    prompt_parts.append("- 2-3 useful custom slash commands for this project type")
if run_diagnostics:
    prompt_parts.append("- Run a quick diagnostic to verify everything works")

if PROJECT_DESCRIPTION:
    prompt_parts.append(f"\n**Project details:** {PROJECT_DESCRIPTION}")

prompt_parts.append("\nGive me a summary of what you set up when done.")

BOOTSTRAP_PROMPT = "\n".join(prompt_parts)

print("=" * 60)
print("COPY THIS PROMPT INTO CLAUDE CODE:")
print("=" * 60)
print()
print(BOOTSTRAP_PROMPT)
print()
print("=" * 60)
print()
print("TERMINAL COMMANDS:")
print("-" * 40)
print("source ~/.bashrc")
print(f'cd "{WORKSPACE_PATH}"')
print("claude")

---

## Reference

### Future Sessions
**Ephemeral mode:** Just re-run this notebook each session.

**Persistent mode:** After first setup, future sessions only need:
```python
from google.colab import drive
drive.mount('/content/drive')
!curl -fsSL https://claude.ai/install.sh | bash
# Then: source ~/.bashrc && cd /content/drive/My\ Drive/claude-workspaces/PROJECT && claude
```

### Useful Commands
| Command | Purpose |
|---------|----------|
| `/init` | Generate CLAUDE.md from codebase |
| `/clear` | Reset conversation context |
| `/compact` | Summarize to save tokens |
| `/model opus` | Switch to Opus |
| `/cost` | Check token usage |
| `#` key | Quick-add to memory |

> **Note:** `/doctor` may hang in Colab's terminal. Use `/cost` to verify Claude works.
